In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
import warnings
warnings.filterwarnings("ignore")
import optuna

In [ ]:
train_data = pd.read_csv("../input/tabular-playground-series-may-2022/train.csv")
test_data = pd.read_csv("../input/tabular-playground-series-may-2022/test.csv")

In [ ]:
print(f"the shape of train data is : {train_data.shape}")
print(f"the shape of test data is : {test_data.shape}")

In [ ]:
# Taking Reference from the beautiful notebook crated by AMBROSM
# Link of the notebook : https://www.kaggle.com/code/ambrosm/tpsmay22-keras-quickstart

features = [f for f in test_data.columns if f != 'id' and f != 'f_27']
float_features = [f for f in features if test_data[f].dtype == float]
for df in [train_data, test_data]:
    # Extract the 10 letters of f_27 into individual features
    for i in range(10):
        df[f'ch{i}'] = df.f_27.str.get(i).apply(ord) - ord('A')
        
    # unique_characters feature is from https://www.kaggle.com/code/cabaxiom/tps-may-22-eda-lgbm-model
    df["unique_characters"] = df.f_27.apply(lambda s: len(set(s)))
    
    # Feature interactions: create three ternary features
    # Every ternary feature can have the values -1, 0 and +1
    df['i_02_21'] = (df.f_21 + df.f_02 > 5.2).astype(int) - (df.f_21 + df.f_02 < -5.3).astype(int)
    df['i_05_22'] = (df.f_22 + df.f_05 > 5.1).astype(int) - (df.f_22 + df.f_05 < -5.4).astype(int)
    i_00_01_26 = df.f_00 + df.f_01 + df.f_26
    df['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.0).astype(int)
    
features = [f for f in test_data.columns if f != 'id' and f != 'f_27']
float_features = [f for f in features if test_data[f].dtype == float]
int_features = [f for f in features if test_data[f].dtype == int and f.startswith('f')]
ch_features = [f for f in features if f.startswith('ch')]

In [ ]:
train_data.head(5)

In [ ]:
test_data.head(5)

In [ ]:
useful_features = [feature for feature in train_data.columns if feature not in ["id", "target","f_27"]]

In [ ]:
def create_nn_model():
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=len(useful_features)))
    model.add(tf.keras.layers.Dense(256,activation="relu",kernel_regularizer = tf.keras.regularizers.l2(30e-6)))
    model.add(tf.keras.layers.Dense(128,activation="relu",kernel_regularizer = tf.keras.regularizers.l2(30e-6)))
    model.add(tf.keras.layers.Dense(128,activation="relu",kernel_regularizer = tf.keras.regularizers.l2(30e-6)))
    model.add(tf.keras.layers.Dense(128,activation="relu",kernel_regularizer = tf.keras.regularizers.l2(30e-6)))
    model.add(tf.keras.layers.Dense(64,activation="relu",kernel_regularizer = tf.keras.regularizers.l2(30e-6)))
    model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
    
    return model

In [ ]:
# model.compile(optimizer="adam",
#               loss=tf.keras.losses.BinaryCrossentropy(),
#               metrics=tf.keras.metrics.AUC())

In [ ]:
def fit_model(X_train,y_train,X_valid,y_valid,test_data, fold):
    
    lr = ReduceLROnPlateau(monitor="val_loss", factor=0.7, 
                               patience=4, verbose=0)
    
    es = EarlyStopping(monitor="val_loss",
                               patience=12, 
                               verbose=1,
                               mode="min", 
                               restore_best_weights=True)
    
    CALLBACKS = [lr, es, tf.keras.callbacks.TerminateOnNaN()]
    
    # Scaling the features using Standard Scaler
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_valid = scaler.transform(X_valid)
    
    model = create_nn_model()
    
    model.compile(optimizer="adam",
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=tf.keras.metrics.AUC())
    
    history = model.fit(X_train,y_train,
                        epochs=200,
                        validation_data=(X_valid,y_valid),
                        batch_size=2048, 
                        callbacks=CALLBACKS,
                        verbose=0)
    
    history_store.append(history.history)
    
    print(f'Training loss after {fold} fold is :{history_store[-1]["loss"][-1]:.3f}')
    
    CALLBACKS, es, lr, tm, history = None, None, None, None, None
    
    y_val_pred = model.predict(X_valid)
    
    validation_auc_score = roc_auc_score(y_valid,y_val_pred)
    
    print(f"Validation ROC_AUC score after {fold} fold is {validation_auc_score}")
    print("="*60)
    
    roc_score.append(validation_auc_score)
    
    test_predictions = model.predict(scaler.fit_transform(test_data.drop(["id","f_27"],axis=1)))
    
    final_predictions.append(test_predictions)   
    

In [ ]:
kf = KFold(n_splits=5,shuffle=True,random_state=42)

history_store = []
    
roc_score = []
    
final_predictions = []

for fold, (trn_idx, val_idx) in enumerate(kf.split(X=train_data)):
    
    X_train, X_val = train_data.iloc[trn_idx][useful_features], train_data.iloc[val_idx][useful_features]
    y_train, y_val = train_data.iloc[trn_idx]["target"], train_data.iloc[val_idx]["target"]
    
    fit_model(X_train, y_train, X_val, y_val,test_data, fold)
    
print(f"the mean validation ROC_AUC score is {np.mean(roc_score)}")

In [ ]:
ids = test_data["id"]

target = np.mean(np.column_stack(final_predictions), axis=1)

In [ ]:
output_data = pd.DataFrame({"id":ids,"target":target})

In [ ]:
output_data

In [ ]:
output_data.to_csv("nn_output_submission13.csv",index=False)

In [ ]:
# lr = ReduceLROnPlateau(monitor="val_loss", factor=0.7, 
#                                patience=4, verbose=0)
# es = EarlyStopping(monitor="val_loss",
#                            patience=12, 
#                            verbose=1,
#                            mode="min", 
#                            restore_best_weights=True)
# CALLBACKS = [lr, es, tf.keras.callbacks.TerminateOnNaN()]

In [ ]:
# # Scaling the features using Standard Scaler
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_valid = scaler.transform(X_valid)

# Finding best parameters for NN using Optuna